<a href="https://colab.research.google.com/github/sadnow/AnimationKit-AI_Upscaling-Interpolation_RIFE-RealESRGAN/blob/testing/TESTING_AnimationKit_Rife_RealESRGAN_Upscaling_Interpolation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AnimationKit AI - Upscaling & Interpolation using Real-ESRGAN+RIFE

***WARNING: THIS IS THE TESTING BRANCH. CHECK OUT MAIN FOR MORE STABILITY***

Credits: Motion smoothing conceived from "Zoom animation processing and motion interpolation" added by https://twitter.com/unltd_dream_co. This part of the script uses [RIFE real-time video interpolation](https://github.com/hzwer/arXiv2020-RIFE) to smooth out the resulting video. 

Upscaling uses Real-ESRGAN (https://github.com/xinntao/Real-ESRGAN). A demo notebook for static images can be found here: https://colab.research.google.com/drive/1k2Zod6kSHEvraybHl50Lys0LerhyTMCo?usp=sharing. The demo was based on the following paper: of our paper [''Real-ESRGAN: Training Real-World Blind Super-Resolution with Pure Synthetic Data''](https://arxiv.org/abs/2107.10833).

<img src="https://raw.githubusercontent.com/xinntao/Real-ESRGAN/master/assets/teaser.jpg" width="100%">

Special thanks to @sportsracer48 and everyone on his Discord. If you want closed beta access to the best VQGAN animation notebook on the planet, check out https://www.patreon.com/sportsracer48

---

Testing notes:

Many ffmpeg examples at https://docs.google.com/document/d/12X_2YoCnPPN7B3OsgX39aYyRF8OF-TVStkFTkKhWrx4/edit

Python modules in Colab: https://medium.com/analytics-vidhya/importing-your-own-python-module-or-python-file-into-colab-3e365f0a35ec

Colab Markdown guide: https://colab.research.google.com/notebooks/markdown_guide.ipynb?hl=es-ni#scrollTo=Lhfnlq1Surtk

---

-[] Split everything into modules

-[] Add blending commands

In [ ]:
# @title Licensed under the MIT License

# Copyright (c) 2021 Katherine Crowson

# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so, subject to the following conditions:

# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.

# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
# OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN
# THE SOFTWARE.

In [1]:
!nvidia-smi

Fri Sep  3 15:19:54 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# P1: Mount Google Drive & Install Libraries

In [ ]:
#I'm leaving these outside AnimationKit.py for transparency

def installPhase():
  %cd /content/
  installESRGAN()
  %cd /content/
  installRIFE()
  #!pip install ffmpeg-python
  %cd /content/
###

def installESRGAN():
  print("Installing libraries for Real-ESRGAN upscaling.")
  !git clone https://github.com/xinntao/Real-ESRGAN.git
  %cd Real-ESRGAN
  !pip install basicsr
  !pip install facexlib
  !pip install gfpgan
  !pip install -r requirements.txt
  !python setup.py develop
  # Download the pre-trained model(s)
  !wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth -P experiments/pretrained_models
  !wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.2.4/RealESRGAN_x4plus_anime_6B.pth -P experiments/pretrained_models
  !wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.1/RealESRGAN_x2plus.pth -P experiments/pretrained_models
  print("Finished Installing libraries for Real-ESRGAN upscaling.")
###

def installRIFE():
  print("Installing libraries for RIFE motion smoothing.")
  !git clone https://github.com/hzwer/arXiv2020-RIFE RIFE
  !gdown --id 1wsQIhHZ3Eg4_AfCXItFKqqyDMB4NS0Yd
  !7z e RIFE_trained_model_HDv2.zip
  !mkdir /content/RIFE/train_log
  !mv *.pkl /content/RIFE/train_log/
  %cd /content/RIFE/
  !gdown --id 1i3xlKb7ax7Y70khcTcuePi6E7crO_dFc
  !pip3 install -r requirements.txt
  print("Done.")
  print("Finsihed Installing libraries for RIFE motion smoothing.")
###
##
#

'''MARKDOWN
#If RIFE (P2) stops for no reason, go to Runtime>Change Runtime type and set the notebook to "high memory"
'''

#Params
mount_google_drive = True #@param {type:"boolean"}

#Mount Google Drive
if mount_google_drive:
  from google.colab import drive
  drive.mount('/content/drive')

#install dependencies
installPhase()

# P2: Real-ESRGAN Video Upscaling (Somewhat Experimental)

Planned additions: Deflickering

In [19]:
import os, sys, re

def splitFrames():
  #Extract Individual frames
  output_file_path = re.search("^[\/].+\/", input_path_mp4)
  output_file_path_raw = output_file_path.group(0)
  delsplit = re.search("\/(?:.(?!\/))+$", input_path_mp4)
  filename = re.sub("^[\/]", "", delsplit.group(0))
  filename_raw = re.sub(".{4}$", "", filename)
  file_extension = re.search(".{3}$", filename)
  file_extension_raw = file_extension.group(0)
  os.environ['inputFile'] = input_path_mp4
  os.environ['outputPath'] = output_file_path_raw
  #os.environ['startTime'] = start_time
  #os.environ['endTime'] = end_time
  os.environ['frameRate'] = target_fps
  os.environ['fileName'] = filename_raw
  os.environ['fileExtension'] = file_extension_raw
  #ffmpeg
  !ffmpeg -hide_banner -i "$inputFile" -vsync 0 $frames_dropbox/frame%04d.png
  #!ffmpeg -hide_banner -i "$inputFile" -r "$frameRate"/1 "/content/Real-ESRGAN/upload"/frame%04d.png
###

def runUpscale():
  #upload images
  import os
  from google.colab import files
  import shutil
  %cd /content/Real-ESRGAN/
  upload_folder = 'upload'
  result_folder = 'results'
#  if os.path.isdir(upload_folder):
#      shutil.rmtree(upload_folder)
#  if os.path.isdir(result_folder):
#      shutil.rmtree(result_folder)
#  os.mkdir(upload_folder)
#  os.mkdir(result_folder)
  #run upscaler
  #!python inference_realesrgan.py --model_path experiments/pretrained_models/RealESRGAN_x4plus.pth --input upload --netscale $scale_value --outscale $scale_value --half --face_enhance
  !python /content/Real-ESRGAN/inference_realesrgan.py --model_path $model_path_value --netscale $scale_value --input $frames_dropbox
###

def rebuildFrames():
  #re-encode video from frames
  #results = '/content/Real-ESRGAN/results'
  #!ffmpeg -r 1/5 -pattern_type glob -i '*.png' -c:v libx264 /content/output.mp4   # x264 video
  %cd /content/Real-ESRGAN/results/
  #!ffmpeg -r $target_fps -pattern_type glob -i '*.png' -c:v libx264 $output_path_mp4   # x264 video
  !ffmpeg -r $target_fps -i "out_%d_out.png" $output_path_mp4
###
##
#

###PARAMS
#frame_rate = "23.976" #@param {type:"string"}
#start_time = "00:00:00.000"
#end_time = "00:00:05.000" #@param {type:"string"}
#@markdown #Input settings
extract_video_file = False #@param {type:"boolean"}
input_path_mp4='/content/drive/MyDrive/VQLIPSE/videos/2blackhole.mp4' #@param {type:"string"}
individual_frames_path='/content/drive/MyDrive/VQLIPSE/images_out/2blackhole' #@param {type:"string"}
#@markdown If you checkboxed `extract_video_file`, `individual_frames_path` will be ignored (and vice-versa).
#@markdown
#@markdown ---
output_path_mp4='/content/drive/MyDrive/VQLIPSE/videos/2blackhole_AnKIT.mp4' #@param {type:"string"}
target_fps='60'#@param {type:"string"}
model_path_value='/content/Real-ESRGAN/experiments/pretrained_models/RealESRGAN_x2plus.pth' #@param ['/content/Real-ESRGAN/experiments/pretrained_models/RealESRGAN_x4plus_anime_6B.pth','/content/Real-ESRGAN/experiments/pretrained_models/RealESRGAN_x4plus.pth','/content/Real-ESRGAN/experiments/pretrained_models/RealESRGAN_x2plus.pth'] {type:"string"}
scale_value="2" #@param [2, 4] {type:"string"}

#@markdown `scale_value` should match the model name, eg 4xplus would use scale value of 4
#@markdown Run RIFE below to fill in empty or duplicate frames. (Double, quadruple etc the length of your video)



#
%cd /content/
print("Cleaning up from last run...")
!rm -rf "/content/Real-ESRGAN/results"
!rm -rf "/content/Real-ESRGAN/upload"
!mkdir "/content/Real-ESRGAN/results"
!mkdir "/content/Real-ESRGAN/upload"

#handling of individual frames checkbox
frames_dropbox = "/content/Real-ESRGAN/upload"
if extract_video_file:
  splitFrames() #converts video to individual png images
else:
  %cd $individual_frames_path
  #!find -maxdepth 1 -iname '*.png' -exec cp {} /content/Real-ESRGAN/upload \;
  print("Copying frames to Real-ESRGAN/upload...")
  !find -maxdepth 1 -name '*.png' -print0 | xargs -0 cp -t $frames_dropbox
  print("Finished copying frames to Real-ESRGAN/upload.")
#
%cd /content/
runUpscale()
rebuildFrames()


/content
Cleaning up from last run...
/content/drive/MyDrive/VQLIPSE/images_out/2blackhole
Copying frames to Real-ESRGAN/upload...
Finished copying frames to Real-ESRGAN/upload.
/content
/content/Real-ESRGAN
Testing 0 out_1
Testing 1 out_10
Testing 2 out_100
Testing 3 out_1000
Testing 4 out_1001
Testing 5 out_1002
Testing 6 out_1003
Testing 7 out_1004
Testing 8 out_1005
Testing 9 out_1006
Testing 10 out_1007
Testing 11 out_1008
Testing 12 out_1009
Testing 13 out_101
Testing 14 out_1010
Testing 15 out_1011
Testing 16 out_1012
Testing 17 out_1013
Testing 18 out_1014
Testing 19 out_1015
Testing 20 out_1016
Testing 21 out_1017
Testing 22 out_1018
Testing 23 out_1019
Testing 24 out_102
Testing 25 out_1020
Testing 26 out_1021
Testing 27 out_1022
Testing 28 out_1023
Testing 29 out_1024
Testing 30 out_1025
Testing 31 out_1026
Testing 32 out_1027
Testing 33 out_1028
Traceback (most recent call last):
  File "/content/Real-ESRGAN/inference_realesrgan.py", line 109, in <module>
    main()
  File 

In [ ]:
import os, sys, re
%cd /content
!rm -rf "/content/Real-ESRGAN/results"
!rm -rf "/content/Real-ESRGAN/upload"
!mkdir "/content/Real-ESRGAN/results"
!mkdir "/content/Real-ESRGAN/upload"
%cd $individual_frames_path
#!find -maxdepth 1 -name '*.png' -print0 | xargs -0 ln -s -t $frames_dropbox
!find -maxdepth 1 -iname '*.png' -exec ln -s {} $frames_dropbox \;

In [15]:
%cd /content/Real-ESRGAN/upload
!ffmpeg -pattern_type glob -i '*.png' -r $target_fps -c:v libx264 $output_path_mp4   # x264 video

/content/Real-ESRGAN/upload
ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 -

# Motion interpolation (RIFE Smoothing)

"(...) (O)utput can be a bit wobbly, so video interpolation can be used to smooth things out a bit."

New: Type in the number of seconds you want your clip to be!


In [13]:
def detect_fps():
  fps_ffprobe = !ffprobe -v error -select_streams v -of default=noprint_wrappers=1:nokey=1 -show_entries stream=avg_frame_rate $input_path
  fps_unfinished = [str(i) for i in fps_ffprobe] # Converting integers into strings
  fps_unfinishedTwo = str("".join(fps_unfinished)) # Join the string values into one string
  numbers = re.findall('[0-9]+', fps_unfinishedTwo)
  newNum = numbers[0:1]
  strings = [str(integer) for integer in newNum]
  a_string = "".join(strings)
  fps = int(a_string)
  #print("Detected FPS is",fps)
  return fps
#

def detect_duration():
  duration_ffprobe = !ffprobe -v error -select_streams v:0 -show_entries stream=duration -of default=noprint_wrappers=1:nokey=1 input.mp4
  duration_unfinished = [str(i) for i in duration_ffprobe] # Converting integers into strings
  duration_unfinishedTwo = str("".join(duration_unfinished)) # Join the string values into one string
  numbers = re.findall('[0-9]+', duration_unfinishedTwo)
  newNum = numbers[0:1]
  strings = [str(integer) for integer in newNum]
  a_string = "".join(strings)
  duration = int(a_string)
  #print("Detected duration INTEGER (in seconds) is",duration)
  return duration
#

def exp_calc():
  import numpy as np
  a = measured_fps * measured_duration
  b = target_fps * length_in_seconds
  c = b / a
  l = np.log(c) / np.log(2)
  print("Un-rounded --exp is",l)
  x = round(l)
  print("Rounding up to an --exp of ",x)
  return x

%cd /content/RIFE/
# @title Settings
#input_path='/content/output.mp4' #@param {type:"string"}
input_path='/content/drive/MyDrive/VQLIPSE/videos/2blackhole_AnKIT.mp4' #@param {type:"string"}

#autodetect fps


target_fps=360#@param {type:"integer"}
length_in_seconds=60#@param {type:"integer"}
#length_multiplier=3#@param {type:"integer"}
#@markdown
#@markdown `length_multiplier` might double, triple, or quadruple your video. Or square it? It's mysterious.
#@markdown
#@markdown ---
#@markdown Note: Your output will look like `original_2x_60fps.mp4` and can be found in your `input_path`.
#@markdown
#@markdown Tip: Save as a high FPS (eg 999) if you plan on doing heavy editing (that way you don't need to re-process at the end.)


measured_fps = detect_fps()
print("Detected average FPS of",input_path,"is",measured_fps)
measured_duration = detect_duration()
print("Detected duration INTEGER (in seconds) is",measured_duration)
exp_value = exp_calc()
print("Warning: Target duration currently rounds to the closest integer.")
!python3 /content/RIFE/inference_video.py --fps=$target_fps --exp=$exp_value --video=$input_path
%cd /content/

/content/RIFE
Detected average FPS of /content/drive/MyDrive/VQLIPSE/videos/2blackhole_AnKIT.mp4 is 60
Detected duration INTEGER (in seconds) is 1
Un-rounded --exp is 8.491853096329676
Rounding up to an --exp of  8
Loaded v2.x HD model.
/content/drive/MyDrive/VQLIPSE/videos/2blackhole_AnKIT.mp4, 1465.0 frames in total, 60.0FPS to 360FPS
Will not merge audio because using png, fps or skip flag!
  3% 41/1465.0 [04:09<2:31:08,  6.37s/it]
/content


# x265 Conversion/Compression (Experimental, Very Fast)

Sort of like Handbrake - good for large filesizes. Can turn 500mb files into 200mb files with negligable quality loss.

**Note: You manually need to input your desired compress_path (sorry) - will update this in the future w/ a checkbox to automatically determine your outputted interpolated file.**

Also, too high of compressions might require a decent local GPU to view. Will add h264 toggle in the future~

In [ ]:
compress_path='/content/drive/MyDrive/VQLIPSE/videos/dedz_akit_4X_360fps.mp4' #@param {type:"string"}
outputStr = '_x265-converted.mp4'
constant_quality=27#@param {type:"slider", min:20, max:50, step:1}


#!ffmpeg -i $compress_path -vcodec hevc_nvenc -rc:v vbr_hq -cq $constant_quality $compress_path$outputStr -async 1 -vsync 1
!ffmpeg -i $compress_path -c:v hevc_nvenc -rc vbr -cq $constant_quality -qmin $constant_quality -qmax $constant_quality -b:v 0 $compress_path$outputStr 

#@markdown Default `constant_quality` is `27`, which scrunched a 500mb 40 min video into ~200mb. Higher values = lower filesize, lower quality

#@markdown Note: Your output will look like `original_2x_60fps.mp4__x265-converted.mp4` and can be found in your `compress_path`.